# Test modules using Polytropes #
Test modules with M, rho, P from polytrope solution and using $P = \rho \frac{R}{\mu}T$ as the equation of state

In [ ]:
%pylab ipympl
ifig = 0

import polytrope
import opacity.opal as opal
import nuclear

# Polytrope and Equation of state #

In [ ]:
n = 3
rho_c = 150 # [g/cm^3]
P_c = 1e17 # [dyn/cm^2]
X, Z = 0.7, 0.02 # mass fraction
data = polytrope.solve_polytrope(rho_c, P_c, n)
m, r, rho, P = polytrope.polytrope_to_cgs(data, rho_c, P_c, n)

R = 8.314e7 # in [erg/(mol K)]
mu = 4/(5*X + 3 - Z) # in [g/mol]

T = P*mu/(rho*R) # in [K]
T_c = T.max()
print(f"Core tempurature: {T_c} in [K]")

rho = numpy.abs(rho)
P = numpy.abs(P)
T = numpy.abs(T)

## opacity ##

In [ ]:
opal_data = opal.read_file("opacity/tables/GN93hz")
opacity = opal.linear_opacity(opal_data)
k_R = opacity(rho, T, X, Z) # in (cm^2/g)
mfp = 1/(rho*k_R)

## nuclear network ##

In [ ]:
N_A = 6.022e23 # in [#/mol]

reactions = nuclear.network.read_network_file("nuclear/networks/pp_branch_I")
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
nuclear.network.draw_network(reactions, show=False)
pyplot.show(ifig)

particles, get_dydt = nuclear.network.build_network(reactions, "./nuclear/reactions")
print(f"particles: {particles}")

Y = 1 - X - Z

X_0 = [Z/(len(particles) - 2)]*len(particles)
X_0[0], X_0[3] = X, Y
X_0[1], X_0[2] = 0*X, 0.1*Y

Y_0 = [x/AZ[0] for x, AZ in zip(X_0, particles)]
epsilon = numpy.zeros(shape=T.shape)
for i in range(len(T)):
    dYdt, epsilon_func = get_dydt(rho[i], T[i])
    epsilon[i] = epsilon_func(0, Y_0)

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(m/m.max(), r/r.max(), label="radius $y = R$")
pyplot.plot(m/m.max(), rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(m/m.max(), P/P.max(), label="pressure $y = P$")
pyplot.plot(m/m.max(), T/T.max(), label="tempurature $y = T$")
pyplot.plot(m/m.max(), k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(m/m.max(), epsilon/epsilon.max(), label="power density $y = \\varepsilon$")
pyplot.xlabel("mass coordinate")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, m/m.max(), label="mass $y = M$")
pyplot.plot(r, rho/rho.max(), label="density $y = \\rho$")
pyplot.plot(r, P/P.max(), label="pressure $y = P$")
pyplot.plot(r, T/T.max(), label="tempurature $y = T$")
pyplot.plot(r, k_R/k_R.max(), label="mean opacity $y = k_R$")
pyplot.plot(r, epsilon/epsilon.max(), label="power density $y = \\varepsilon$")
pyplot.xlabel("radius [cm]")
pyplot.ylabel("parameter $y$ fraction of maximum")
pyplot.legend()
pyplot.show(ifig)

In [ ]:
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.semilogy(r, mfp)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("mean free path [cm]")
pyplot.title("Opacity: Mean free path of light")
pyplot.show(ifig)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
pyplot.plot(r, epsilon*1.602e-6*0.1)
pyplot.xlabel("radius [cm]")
pyplot.ylabel("power density [W/m^3]")
pyplot.title("Nuclear power density")
pyplot.show(ifig)